# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets.

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.

Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:*

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd


In [3]:
# your code here

# Replace the path with the correct file path to your CSV file
file_path = '/content/drive/MyDrive/Ironhack Week_8_Day_3_Morning/training.1600000.processed.noemoticon.csv'

# Load the data into a DataFrame using pd.read_csv and specify the encoding
df = pd.read_csv(file_path, encoding='latin-1')

# Display the first few rows using the head() method
df.head()


,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [4]:
# Define the column names as mentioned: ['target', 'id', 'date', 'flag', 'user', 'text'].

df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
df.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [5]:
sample_size = 20000  # Define the desired sample size
df = df.sample(n=sample_size, random_state=42)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [6]:
!pip install nltk

In [7]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download the 'punkt', 'wordnet', and 'stopwords' resources from NLTK
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


# Function to clean up the text data
def clean_up(s):
    s = re.sub(r'\d+', ' ',s)
    s = re.sub(r'http\S+', ' ',s)
    s = re.sub(r'[^\w\s]', ' ',s)
    s = s.lower()
    return s

# Function to tokenize the text data
def tokenize(s):
    s = word_tokenize(s)
    return s

# Function to remove stopwords from the tokens
def remove_stopwords(l):
    lemmatized = [wnl.lemmatize(word) for word in l]
    without_sw = [word for word in lemmatized if word not in stopwords.words()]
    return without_sw

df['text_processed'] = df['text'].apply(clean_up)
df['text_processed'] = df['text_processed'].apply(tokenize)

ps = PorterStemmer()
wnl = WordNetLemmatizer()

# Function to stem and lemmatize the tokens
def stem_and_lemmatize(l):
    stemmed = [ps.stem(w) for w in l]
    lemmatized = [wnl.lemmatize(word) for word in stemmed]
    return lemmatized

df['text_processed'] = df['text_processed'].apply(stem_and_lemmatize)
df['text_processed'] = df['text_processed'].apply(remove_stopwords)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
df.head()


,target,id,date,flag,user,text,text_processed
541200,0,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,@Nkluvr4eva My poor little dumpling In Holmde...,"[nkluvr, eva, poor, littl, dumpl, holmdel, wa,..."
750,0,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,I'm off too bed. I gotta wake up hella early t...,"[bed, wake, hella, earli, tomorrow, morn]"
766711,0,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,I havent been able to listen to it yet My spe...,"[abl, listen, speaker, bust]"
285055,0,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...,"[rememb, whi, solv, rel, big, equat, unknown, ..."
705995,0,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,"Ate too much, feel sick","[ate, feel, sick]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [9]:
# your code here
# Import the necessary libraries
from nltk import FreqDist
from nltk import NaiveBayesClassifier

# Combine all the words in the text_processed column
all_words = [word for sublist in df['text_processed'] for word in sublist]

all_words

['nkluvr',
 'eva',
 'poor',
 'littl',
 'dumpl',
 'holmdel',
 'wa',
 'realli',
 'hope',
 'hard',
 'tonight',
 'xx',
 'bed',
 'wake',
 'hella',
 'earli',
 'tomorrow',
 'morn',
 'abl',
 'listen',
 'speaker',
 'bust',
 'rememb',
 'whi',
 'solv',
 'rel',
 'big',
 'equat',
 'unknown',
 'total',
 'pain',
 'butt',
 'ate',
 'feel',
 'sick',
 'purchas',
 'park',
 'domain',
 'godaddi',
 'drain',
 'kind',
 'gambl',
 'lunch',
 'dj',
 'eat',
 'back',
 'tech',
 'equin',
 'medic',
 'center',
 'poor',
 'lilli',
 'stall',
 'rest',
 'anoth',
 'month',
 'twitter',
 'account',
 'super',
 'bum',
 'tamaraschil',
 'adventur',
 'life',
 'glad',
 'great',
 'week',
 'yr',
 'respons',
 'perezhilton',
 'zach',
 'make',
 'pee',
 'grown',
 'gay',
 'sum',
 'day',
 'word',
 'kacker',
 'wkj',
 'great',
 'mind',
 'alik',
 'veri',
 'tire',
 'clean',
 'anymor',
 'lochnesscullen',
 'follow',
 'nessi',
 'sallygreen',
 'andrewtc',
 'syond',
 'guy',
 'coupl',
 'love',
 'handl',
 'inch',
 'tayyab',
 'feel',
 'quit',
 'sleepi',

In [10]:
# Calculate the frequency distribution of all words
freq_dist = FreqDist(all_words)

freq_dist

FreqDist({'wa': 1400, 'day': 1324, 'work': 1122, 'love': 1114, 'quot': 942, 'today': 829, 'time': 795, 'miss': 709, 'back': 664, 'feel': 657, ...})

In [11]:
#Select the top 5,000 words from the frequency distribution
top_words = freq_dist.most_common(5000)

top_words

[('wa', 1400),
 ('day', 1324),
 ('work', 1122),
 ('love', 1114),
 ('quot', 942),
 ('today', 829),
 ('time', 795),
 ('miss', 709),
 ('back', 664),
 ('feel', 657),
 ('realli', 605),
 ('night', 582),
 ('amp', 574),
 ('make', 551),
 ('watch', 536),
 ('home', 507),
 ('hope', 496),
 ('morn', 445),
 ('great', 438),
 ('wait', 423),
 ('tomorrow', 421),
 ('twitter', 418),
 ('wish', 416),
 ('sleep', 382),
 ('haha', 379),
 ('whi', 375),
 ('follow', 369),
 ('sad', 366),
 ('veri', 351),
 ('fun', 348),
 ('nice', 335),
 ('sorri', 332),
 ('happi', 330),
 ('week', 325),
 ('tonight', 322),
 ('onli', 315),
 ('hate', 309),
 ('friend', 308),
 ('gon', 299),
 ('start', 290),
 ('show', 281),
 ('bed', 278),
 ('lt', 268),
 ('peopl', 264),
 ('tweet', 263),
 ('hour', 259),
 ('final', 246),
 ('weekend', 240),
 ('play', 239),
 ('school', 236),
 ('littl', 234),
 ('everyon', 230),
 ('awesom', 229),
 ('sick', 228),
 ('guy', 224),
 ('hey', 221),
 ('life', 217),
 ('plea', 217),
 ('movi', 216),
 ('girl', 213),
 ('alway', 

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [12]:
df.columns

Index(['target', 'id', 'date', 'flag', 'user', 'text', 'text_processed'], dtype='object')

In [13]:
# Create a list of tuples containing the bag-of-words features and the target output
features = [
    (
        {word: (word in tweet) for word in top_words},
        (sentiment == 4)
    )
    for tweet, sentiment in zip(df['text_processed'], df['target'])
]

# Print an example of the first feature set
print(features[0])

({('wa', 1400): False, ('day', 1324): False, ('work', 1122): False, ('love', 1114): False, ('quot', 942): False, ('today', 829): False, ('time', 795): False, ('miss', 709): False, ('back', 664): False, ('feel', 657): False, ('realli', 605): False, ('night', 582): False, ('amp', 574): False, ('make', 551): False, ('watch', 536): False, ('home', 507): False, ('hope', 496): False, ('morn', 445): False, ('great', 438): False, ('wait', 423): False, ('tomorrow', 421): False, ('twitter', 418): False, ('wish', 416): False, ('sleep', 382): False, ('haha', 379): False, ('whi', 375): False, ('follow', 369): False, ('sad', 366): False, ('veri', 351): False, ('fun', 348): False, ('nice', 335): False, ('sorri', 332): False, ('happi', 330): False, ('week', 325): False, ('tonight', 322): False, ('onli', 315): False, ('hate', 309): False, ('friend', 308): False, ('gon', 299): False, ('start', 290): False, ('show', 281): False, ('bed', 278): False, ('lt', 268): False, ('peopl', 264): False, ('tweet', 26

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [14]:
# your code here
import random
from nltk import NaiveBayesClassifier
from nltk.classify import accuracy

# Split the feature set into a training set and a test set
train_size = int(0.8 * len(features))
train_set = features[:train_size]
test_set = features[train_size:]

# Create a Naive Bayes classifier and train it with the training set
classifier = NaiveBayesClassifier.train(train_set)

# Print the most informative features
classifier.show_most_informative_features()

# Evaluate the accuracy of the classifier on the test set
accuracy = accuracy(classifier, test_set)
print("Accuracy:", accuracy)

Most Informative Features
               ('_', 27) = False           False : True   =      1.0 : 1.0
              ('___', 3) = False           False : True   =      1.0 : 1.0
               ('aa', 5) = False           False : True   =      1.0 : 1.0
          ('aaaaaah', 3) = False           False : True   =      1.0 : 1.0
             ('aaah', 5) = False           False : True   =      1.0 : 1.0
             ('aagw', 2) = False           False : True   =      1.0 : 1.0
              ('aah', 4) = False           False : True   =      1.0 : 1.0
             ('aahh', 3) = False           False : True   =      1.0 : 1.0
            ('aaron', 4) = False           False : True   =      1.0 : 1.0
          ('abandon', 3) = False           False : True   =      1.0 : 1.0
Accuracy: 0.51


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [15]:
# your code here
from nltk.classify import accuracy

# Test the classifier on the test set and calculate the accuracy score
accuracy_score = accuracy(classifier, test_set)

# Print the accuracy score
print("Accuracy:", accuracy_score)

Accuracy: 0.51


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [16]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time.

In [17]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [18]:
# your code here